In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import torch
import kornia
import numpy as np
import gc
import os
import sys
%matplotlib notebook
import matplotlib.pyplot as plt

pwd = os.path.abspath(os.path.join('.'))
pwd = pwd.split('/')
pwd = pwd[0:-1]
pwd.append('src')

module_path = '/'.join(pwd)
if module_path not in sys.path:
    sys.path.append(module_path)

from helpers import Trainer
from helpers import Logger
from helpers import Trial
from mibi_dataloader import FlatMIBIData
from modules import Distiller
from criteria import DistillerLoss
import utils

In [3]:
pixs = FlatMIBIData(folder='/Volumes/G-DRIVE USB/matpoints/')

Point1.mat
